#**CARGA DE LIBRERIAS**

In [ ]:
%%time
!pip uninstall pillow -y --q
!pip install pillow==10.0.1 --q
!pip install langchain-community --q
!pip install langchain --q
!pip install langchain_openai --q
!pip install "unstructured[pdf]"==0.11.8 --q
!pip install sentence-transformers --q
!pip install faiss-cpu --q
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import Anyscale
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
import os
os.environ["OPENAI_BASE_URL"] = userdata.get('OPENAI_BASE_URL')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
!pip install pandas==1.5.3
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
!pip install gradio==4.14.0 --q
import gradio as gr
!pip install datasets==2.16.1 --q
from datasets import Dataset
#SE CARGA EL MODELO DE EMBEDDINGS
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#SE DEFINE EL LLM A UTILIZAR Y SU TEMPERATURA
llm = ChatOpenAI(model_name="meta-llama/Llama-2-70b-chat-hf", temperature=0)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 11.9 s, sys: 1.7 s, total: 13.6 s
Wall time: 3min 59s


#**EJECUTAR EL PROTOTIPO**

In [ ]:
def cargar_procesar_configurar_historias_y_responder_preguntas(archivo_xls, pdf_path, pregunta1, pregunta2, pregunta3, pregunta4, pregunta5):
    # Cargar datos desde el archivo Excel, eliminando los saltos de pagina.
    df = pd.read_excel(archivo_xls, sheet_name='Sheet1')
    df['DATOS'] = df['DATOS'].astype(str).apply(openpyxl.utils.escape.unescape)
    df['DATOS'] = df['DATOS'].str.replace('\r\n', '')
    df['DATOS'] = df['DATOS'].str.replace('\n', '')
    df1 = df[['ID','DATOS']]
    dataset = Dataset.from_pandas(df1)

    # Cargar el PDF y dividirlo en chunks
    loader = UnstructuredPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=50,length_function=len,is_separator_regex=False,)
    text_chunks = text_splitter.split_documents(documents)

    # Crear la base de datos vectorial ejecutando el modelo de embeddings sobre los chunks
    vectorstore = FAISS.from_documents(text_chunks, embeddings)



    # Establecer el prompt template para ajustar las respuestas recibidas
    template ="""Usa las siguientes piezas de contexto para responder a las preguntas que se presentaran usando solamente SI o NO. No es necesario ampliar el por qué de la respuesta. No es necesario dar el motivo de la respuesta. NO es necesario dar un resumen de la historia. No es necesario dar un resumen de la respuesta.
    Solamente estás autorizado para responder usando SI o NO. No puedes usar ningún otro tipo de texto para responder. No es necesario agregar nada más que SI o NO. No es necesario usar signos de puntuación como . , ; o similares.
    Tómate el tiempo para pensar las respuestas y consultar el contexto antes de responder de forma apresurada.
    {contexto}

    Pregunta: {question}"""

    custom_rag_prompt = PromptTemplate.from_template(template)
    retriever=vectorstore.as_retriever()
    rag_chain = (
        {"contexto": retriever, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )


    # Configurar QA con el prompt template
    qa = rag_chain

    # Crear un nuevo DataFrame para almacenar las respuestas
    df_respuestas = pd.DataFrame(columns=['ID', 'Respuesta1', 'Respuesta2', 'Respuesta3', 'Respuesta4', 'Respuesta5'])

    # Iterar a través de los índices del dataset
    for index in range(len(dataset)):
        # Obtener la historia clínica de la fila actual
        historia_clinica = dataset['DATOS'][index]

        # Hacer la pregunta al sistema y obtener la respuesta
        respuestas = []
        for pregunta in [pregunta1, pregunta2, pregunta3, pregunta4, pregunta5]:
            respuesta = qa.invoke(f"En {historia_clinica} {pregunta}").strip()
            respuestas.append(respuesta)

        # Agregar la respuesta al nuevo DataFrame
        df_respuestas = df_respuestas.append({
            'ID': dataset['ID'][index],
            'Respuesta1':respuestas[0],
            'Respuesta2':respuestas[1],
            'Respuesta3':respuestas[2],
            'Respuesta4':respuestas[3],
            'Respuesta5':respuestas[4]
        }, ignore_index=True)

    return df_respuestas



##función para trasladar el df de respuestas
def minuscula(df_respuestas):
    # cargar el DF
    df_respuestas = pd.DataFrame(df_respuestas)

    return df_respuestas

##funcion para convertir el df en excel exportable
def export_excel(df_respuestas):
    df_respuestas.to_excel("output.xlsx")
    return gr.File(value="output.xlsx", visible=True)


##funcion para convertir el df a 1 y 0
def limpiar_dataframe(df_respuestas):

    # Iterar sobre todas las columnas excepto la primera
    for columna in df_respuestas.columns[1:]:
        # Función para convertir 'SI' en 1 y 'NO' en 0
        def convertir_si_no(valor):
            if valor.startswith('SI'):
                return 1
            elif valor.startswith('NO'):
                return 0
            else:
                return valor

        # Aplicar la función a todas las celdas de la columna actual
        df_respuestas[columna] = df_respuestas[columna].apply(convertir_si_no)

    # Agregar una columna con el promedio de cada fila
    df_respuestas['Promedio'] = (df_respuestas[df_respuestas.columns[1:]].mean(axis=1))
    return df_respuestas


###funcion para calcular adherencia global

def promedio_por_columna(df_respuestas):
    # Calcular el promedio de cada columna, excluyendo la columna 'DOC'
    promedio_por_columna = df_respuestas[df_respuestas.columns[1:]].mean(axis=0)
    promedio_por_columna *= 100
    # Convertir la Serie en un DataFrame
    promedio_df = pd.DataFrame(promedio_por_columna, columns=['Promedio']).reset_index()
    promedio_df.columns = ['Pregunta', 'Promedio']
    return promedio_df


###funcion para graficar la adherencia

def GRAFICO_BARRAS(promedio_df):
    fig, ax1 = plt.subplots(figsize=(10, 6))
    promedio_df.set_index('Pregunta').plot(kind='bar', ax=ax1, legend=False)
    ax1.set_ylabel('Porcentaje de Adherencia')
    ax1.set_title('Adherencia por Respuesta')
    plt.xticks(rotation=0)

    # Agregar etiquetas de datos encima de cada columna
    for p in ax1.patches:
        height = p.get_height()
        ax1.annotate(f'{height:.2f}%', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')


    return fig


######### GENERANDO LA INTERFAZ GRAFICA EN DOS PESTAÑAS Y ACORDEONES SELECCIONADOS, SE DEJAN LAS PREGUNTAS TRAZADORAS SELECCIONADAS


with gr.Blocks() as demo:
    with gr.Tab("PROCESAR EHR Y GPC"):
      gr.HTML("<p style='text-align:center'><b>INGRESE AQUÍ LA BASE DE DATOS CON HISTORIAS CLÍNICAS EN FORMATO XLS Y LA GPC EN FORMATO PDF</b></p>")
      with gr.Accordion("INGRESAR BDD Y GPC", open=False):
          with gr.Row():
              inp_xls = gr.File(label="Archivo XLS")
              inp_pdf = gr.File(label="Archivo PDF")
      with gr.Accordion("REVISAR PREGUNTAS A EVALUAR", open=False):
          with gr.Row():
              inp_pregunta1 = gr.Textbox(label="Pregunta 1", value='La descripción del examen físico de la próstata, ¿es adecuada según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta2 = gr.Textbox(label="Pregunta 2", value='El tratamiento propuesto para el paciente ¿es adecuado según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta3 = gr.Textbox(label="Pregunta 3", value='La descripcion del cuadro clinico del paciente ¿es adecuada y completa según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta4 = gr.Textbox(label="Pregunta 4", value='¿Se le entregaron al paciente recomendaciones y signos de alarma de acuerdo a lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta5 = gr.Textbox(label="Pregunta 5", value='¿Los paraclinicos solicitados fueron adecuados según lo especificado en la guía? SI o NO.', lines = 4)
      with gr.Accordion("RESULTADOS", open=False):
        outRespuestas = gr.Dataframe(label="Respuestas", visible= True, interactive= False)
      btn_procesar = gr.Button("PROCESAR EHR Y GPC")
      btn_procesar.click(cargar_procesar_configurar_historias_y_responder_preguntas,inputs=[inp_xls, inp_pdf, inp_pregunta1, inp_pregunta2, inp_pregunta3, inp_pregunta4, inp_pregunta5],outputs=outRespuestas)



    with gr.Tab("EVALUAR Y GRAFICAR"):
      gr.HTML("<p style='text-align:center'><b>AQUI SE VISUALIZA LA ADHERENCIA A LA GUIA SELECCIONADA</b></p>")
      with gr.Accordion("VISUALIZAR Y DESCARGAR RESULTADOS", open=False):
          with gr.Row():
              input = outRespuestas ##outRespuestas
              output = gr.Dataframe(label="Respuestas", visible= True)

          btn_procesar = gr.Button("VISUALIZAR RESULTADOS OBTENIDOS")
          btn_procesar.click(minuscula,inputs=input,outputs=output)
          button = gr.Button("DESCARGAR RESULTADOS")
          excel = gr.File(interactive=False, visible=False)
          button.click(export_excel, outRespuestas, excel)

      with gr.Accordion("TRANSFORMAR RESULTADOS", open=False):
          with gr.Row():
              input = outRespuestas ##outRespuestas
              limpios = gr.Dataframe(label="Respuestas", visible= True, interactive=False)

          btn_procesar = gr.Button("TRANSFORMAR RESULTADOS")
          btn_procesar.click(limpiar_dataframe,inputs=input,outputs=limpios)

      with gr.Accordion("CALCULAR ADHERENCIA GLOBAL", open=False):
          with gr.Row():
              input = limpios ##outRespuestas
              promedio_df = gr.Dataframe(label="Respuestas", visible= True, interactive=False)

          btn_procesar = gr.Button("CALCULAR ADHERENCIA GLOBAL")
          btn_procesar.click(promedio_por_columna,inputs=input,outputs=promedio_df)
          button = gr.Button("DESCARGAR RESULTADOS")
          excel = gr.File(interactive=False, visible=False)
          button.click(export_excel, promedio_df, excel)


      with gr.Accordion("GRAFICAR", open=False):
          with gr.Row():
              input = promedio_df
              output = gr.Plot(show_label= True)
          btn_procesar = gr.Button("GRAFICAR ADHERENCIA GLOBAL Y POR CRITERIOS")
          btn_procesar.click(GRAFICO_BARRAS,inputs=input,outputs=output)


demo.launch(share=True, debug=True)

IMPORTANT: You are using gradio version 4.14.0, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2e21a1a052383928e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
<ipython-input-2-bea40aed37dc>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_respuestas = df_respuestas.append({
<ipython-input-2-bea40aed37dc>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_respuestas = df_respuestas.append({
<ipython-input-2-bea40aed37dc>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_respuestas = df_respuestas.append({
<ipython-input-2-bea40aed37dc>:58: FutureWarning: The frame.append method is deprecated and will be remove